## RSSI sample

In [1]:
import math
class RSSISample:
    #constroctor and only leaving two floating numbers in rssi
    def __init__(self, mac_address: str, rssi: float) -> None:
        self.mac_address = mac_address
        self.rssi = math.floor(rssi*100)/100 
    #printing function 
    def printing(self):
        strin= str(self.mac_address)+","+str(self.rssi)
        return strin

## Fingerprint sample


In [2]:
from math import log10
class FingerprintSample:
    #initilize
    def __init__(self, samples: list[RSSISample]) -> None:
        self.samples = samples
        
    #printing function
    def printing(self):
        strin=""
        for l in self.samples:
            strin=strin+","+l.printing()
        return strin

## Location

In [22]:
class SimpleLocation:
    #initilize
    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z
        
    #defining funtion to be able to compare two objects
    def __eq__(self, pos):
        return self.x==pos.x and self.y==pos.y and self.z==pos.z
    def __hash__(self):
        return hash((self.x, self.y, self.z))
    def __ne__(self, other):
        return not(self == other)
  

    #printing function
    def printing(self):
        strin=str(self.x)+","+str(self.y)+","+str(self.z)
        return strin
    

## Fingerprint

In [4]:
class Fingerprint:
    
    #initilize
    def __init__(self, position: SimpleLocation, sample: FingerprintSample) -> None:
        self.position = position
        self.sample = sample
        
    #printing function
    def printing(self):
        print(str(self.position.printing())+",0"+str(self.sample.printing()))

## Fingerprint database


In [5]:
import csv
import math
from math import log10

class FingerprintDatabase:
    
    #initilize
    def __init__(self) -> None:
        #the db where i read from csv in will be a dictionary
        self.db = {}
        #the result list will be saved in here, it is a list of fingerprints
        self.res=[]
        
    #this function will read from csv, it pretty complex because it will read into a 
    #dictionary and check for mac and position duplications
    def readingCSVdata(self):
        with open('data.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
        for i in range(len(data)): 
            pos= SimpleLocation(float(data[i][0]),float(data[i][1]),float(data[i][2]))
            if(not(pos in self.db.keys())):
                j=4
                address={}
                while(j<len(data[i])):
                    if(data[i][j] in address.keys()):
                        address[data[i][j]].append(data[i][j+1])
                    else:
                        lis =[data[i][j+1]]
                        address.update({data[i][j]: lis})
                    j=j+2
                self.db.update({pos: address})
            else:
                j=4
                while(j<len(data[i])):
                    if(data[i][j] in self.db[pos].values()):
                        self.db[pos][data[i][j]].append(data[i][j+1])
                    else:
                        lis =[data[i][j+1]]
                        self.db[pos].update({data[i][j]: lis})
                    j=j+2
                self.db.update({pos: address})
    
    #this function is responsible for the average calculations
    def rssiValcheck(self):
        for i in self.db.values():
            for x in i :
                try:
                    lis=i[x] 
                    if len(lis)>1:
                        somme = 0
                        for j in lis:
                            somme=somme + 10**(float(j)/ 10.)
                        avg=somme/len(lis)
                        val= 10. * math.log10(avg)
                        i.update({x: val})
                    else: 
                        i.update({x: float(lis[0])})      
                except:
                    pass
                       
     #this function is the one responsible for switching from dictionary db to res list              
    def resultarray(self):
        for x in self.db:
            y=self.db[x]
            rssisamp=[]
            for i in y:
                rssisamp.append(RSSISample(i,y[i]))
            fin=FingerprintSample(rssisamp)
            self.res.append(Fingerprint(x,fin)) 
      
    #this function is the one responsible for checking None objects and deleting them from list and same time printing
    def printing(self):
        for r in self.res:
            if(r.printing() == None):
                self.res.remove(r)
            else:
                print(r.printing())

## TD1

Now let check the result in our main file

In [53]:
hi = FingerprintDatabase()
hi.readingCSVdata()
hi.rssiValcheck()
hi.resultarray()
hi.printing()

5.4,29.59,1.2,0,00:13:ce:8f:77:43,-61.0,00:13:ce:97:78:79,-75.0,00:13:ce:8f:78:d9,-28.0,00:13:ce:95:de:7e,-78.0,00:13:ce:95:e1:6f,-67.0
3.0,29.59,1.2,0,00:13:ce:97:78:79,-72.0,00:13:ce:95:de:7e,-76.0,00:13:ce:8f:77:43,-69.0,00:13:ce:95:e1:6f,-59.0,00:13:ce:8f:78:d9,-26.0
5.4,30.59,1.2,0,00:13:ce:95:e1:6f,-52.0,00:13:ce:8f:78:d9,-37.0,00:13:ce:8f:77:43,-69.0,00:13:ce:95:de:7e,-74.0,00:13:ce:97:78:79,-84.0
3.0,30.59,1.2,0,00:13:ce:97:78:79,-77.0,00:13:ce:8f:77:43,-67.0,00:13:ce:8f:78:d9,-28.0,00:13:ce:95:e1:6f,-58.0,00:13:ce:95:de:7e,-73.0
5.4,31.59,1.2,0,00:13:ce:95:e1:6f,-53.0,00:13:ce:8f:77:43,-71.0,00:13:ce:8f:78:d9,-34.0,00:13:ce:95:de:7e,-73.0,00:13:ce:97:78:79,-74.0
3.0,31.59,1.2,0,00:13:ce:8f:77:43,-71.0,00:13:ce:8f:78:d9,-37.0,00:13:ce:95:e1:6f,-68.0
5.4,26.04,1.2,0,00:13:ce:8f:78:d9,-34.0,00:13:ce:8f:77:43,-64.0,00:13:ce:95:e1:6f,-59.0,00:13:ce:95:de:7e,-70.0,00:13:ce:97:78:79,-79.0
3.0,26.04,1.2,0,00:13:ce:95:e1:6f,-49.0,00:13:ce:8f:78:d9,-39.0,00:13:ce:8f:77:43,-75.0,00:13:ce

# ------------------------------------------------------------------------------------------------------------

## Access Points data


In [27]:
class AccessPoint:
    def __init__(self, mac: str, loc: SimpleLocation,  f: float, a: float, p: float):
        self.mac_address = mac
        self.location = loc
        self.output_power_dbm = p
        self.antenna_dbi = a
        self.output_frequency_hz = f
  
    def distanceTo(self, point):
        return math.sqrt((point[0] - self.location.x) ** 2 + (point[1] - self.location.y) ** 2 + (point[2] - self.location.z) ** 2)
    
        

In [28]:
AP = {"00:13:ce:95:e1:6f": AccessPoint("00:13:ce:95:e1:6f", SimpleLocation(4.93, 25.81, 3.55), 2417000000, 5.0, 20.0),
      "00:13:ce:95:de:7e": AccessPoint("00:13:ce:95:de:7e", SimpleLocation(4.83, 10.88, 3.78), 2417000000, 5.0, 20.0), 
      "00:13:ce:97:78:79": AccessPoint("00:13:ce:97:78:79", SimpleLocation(20.05, 28.31, 3.74), 2417000000, 5.0, 20.0), 
      "00:13:ce:8f:77:43": AccessPoint("00:13:ce:8f:77:43", SimpleLocation(4.13, 7.085, 0.80), 2417000000, 5.0, 20.0), 
      "00:13:ce:8f:78:d9": AccessPoint("00:13:ce:8f:78:d9", SimpleLocation(5.74, 30.35, 2.04), 2417000000, 5.0, 20.0)}

In [54]:
import math
import numpy as np
import scipy.optimize as optim

def compute_FBCM_index(distance: float, rssi_values: RSSISample, ap: AccessPoint) -> float:
    wavelen= 3e8 / ap.output_frequency_hz
    wavelenlog=20 * math.log10(wavelen/ 4 * math.pi)
    formula=( ap.output_power_dbm - rssi_values.rssi + ap.antenna_dbi + 2.1 + wavelenlog)/(10 * math.log10(distance))
    return formula

    
def estimate_distance(rssi_avg: float, fbcm_index: float, ap: AccessPoint) -> float:
    wavelen= 3e8 / ap.output_frequency_hz
    pr= ( ap.output_power_dbm - rssi_avg + ap.antenna_dbi + 2.1 + 20*math.log10(wavelen) - 20*math.log10(4*math.pi))/(10.0 * fbcm_index)
    distance =pow(10,pr)
    return distance

        
def multilateration(distances: dict[str, float], ap_locations: dict[str, SimpleLocation]) -> SimpleLocation:
    location = optim.least_squares(lambda x: [(ap_locations[i].distanceTo(distances[x]) - distances[i]) for i in ap_locations],
                                   np.divide(np.sum([(ap_locations[l].location.x, ap_locations[l].location.y, ap_locations[l].location.z) for l in ap_locations], axis=0), len(ap_locations)),
                                   method='lm').x
    return SimpleLocation(location)


In [55]:
dist=round(estimate_distance(-57.0, 3.5, AP["00:13:ce:95:e1:6f"]),3)
print(dist)
rs=RSSISample("00:13:ce:95:e1:6f",-100.0)
i= round(compute_FBCM_index(dist, rs, AP["00:13:ce:95:e1:6f"]),3)
print(i)
dist=round(estimate_distance(rs.rssi,round(i,3), AP["00:13:ce:95:e1:6f"]),3)
print(dist)
#multilateration({"00:13:ce:95:e1:6f",6},AP)

18.069
8.503
10.546


# ------------------------------------------------------------------------------------------------------------

## Simple matching

In [56]:
from statistics import mean
def rssi_distance(sample1: dict[str, float], sample2: dict[str, float]) -> float:
    somme=[]
    for i in sample1:
        somme.append(abs(abs(sample1[i])-abs(sample2[i])))
    return sum(somme)

def simple_matching(db: FingerprintDatabase, sample: dict[str, float]) -> SimpleLocation:
    mindist=1000
    pos=0
    for i in db.db:
        dist=rssi_distance(sample,db.db[i])
        if(dist<mindist):
            mindist=dist
            pos=i
    return pos

## Histogram matching


In [18]:
class NormHisto:
    def __init__(self, histo: dict[int, float]):
        self.histogram = histo
        self.normalized_histogram = {}
        self.normalize()
    def normalize(self):
        # normalize the histogram
        sum_histo = 0
        for i in self.histogram:
            sum_histo += self.histogram[i]
        for i in self.histogram:
            self.normalized_histogram[i] = self.histogram[i] / sum_histo
    def get_normalized_histogram(self):
        return self.normalized_histogram
    def get_histogram(self):
        return self.histogram

In [20]:
def probability(histo1: NormHisto, histo2: NormHisto) -> float:
    # Your code
    pass

def histogram_matching(db: FingerprintDatabase, sample: NormHisto) -> SimpleLocation:
    # Your code (use probability)
    pass

## Gauss matching


In [37]:
class GaussModel:
    def __init__(self, avg: float, stddev: float):
        self.average_rssi = avg
        self.standard_deviation = stddev

In [22]:
def histogram_from_gauss(sample: GaussModel) -> RSSISample:
    # Your code
    pass

## Lab


In [57]:
from flask import Flask
app = Flask(__name__)
@app.route('/')
def hello():
    return 'Hello, World!'